In [2]:
# -*- coding: utf-8 -*-
"""workshop.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1IaKakVo5pdbx4WBv_0UP0630mWox-PJa
"""

# -*- coding: utf-8 -*-
"""main_workshop.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1BMwVOdv2tTZTHteG5jF6DzS0f0VY6XYY
"""

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, TensorDataset
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from typing import Sequence, Tuple
import pandas as pd
from sklearn.datasets import fetch_openml
from collections import OrderedDict, defaultdict
import torch.nn.functional as F
model_equations = []

def fed_model(testimages):
    # Load the Wine Quality dataset
    wine = fetch_openml(name='wine-quality-red', version=1, as_frame=True, parser='liac-arff')
    X = wine.data
    y = wine.target
    y = wine.target.astype(int)  # Ensure the target values are integers

    # Adjust target values: map 3 to 0, 9 to 6, and shift other values accordingly
    def adjust_target(value):
        if value == 3:
            return 0
        elif value == 9:
            return 6
        else:
            return value - 3

    y = y.apply(adjust_target)
    # Standardize the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train = y_train.astype(int)
    y_train_np = y_train.to_numpy().astype(int)
    y_train_tensor = torch.tensor(y_train_np, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test = y_test.astype(int)
    y_test_np = y_test.to_numpy()
    y_test_tensor = torch.tensor(y_test_np, dtype=torch.long)

    # Create TensorDataset and DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # Number of clients
    n_clients = 3

    # Split the training data into n_clients parts
    indices = np.arange(len(train_dataset))
    np.random.shuffle(indices)
    split_indices = np.array_split(indices, n_clients)

    # Create data loaders for each client
    client_loaders = []
    batch_size = 16
    for client_indices in split_indices:
        client_subset = Subset(train_dataset, client_indices)
        client_loader = DataLoader(client_subset, batch_size=batch_size, shuffle=True)
        client_loaders.append(client_loader)

    # Define a simple feedforward neural network model
    class SimpleNN(nn.Module):
        def __init__(self):
            super(SimpleNN, self).__init__()
            self.fc1 = nn.Linear(11, 50)  # Adjust input size to match the number of features (11)
            self.fc2 = nn.Linear(50, 20)
            self.fc3 = nn.Linear(20, 6)  # Adjust output size to match the number of classes (6)
            self.relu = nn.ReLU()

        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.relu(self.fc2(x))
            x = self.fc3(x)
            return x

    # Train the model on each client's data and save the weights
    client_models = []
    epochs = 20
    criterion = nn.CrossEntropyLoss()

    for i, loader in enumerate(client_loaders):
        model = SimpleNN()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        # Training loop
        for epoch in range(epochs):
            running_loss = 0.0
            for inputs, labels in loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            # print(f'Client {i+1}, Epoch {epoch+1}, Loss: {running_loss / len(loader)}')

        # Save the model weights
        torch.save(model.state_dict(), f'client_{i+1}_model.pth')
        client_models.append(model.state_dict())
        sums = defaultdict(int)
        count = len(client_models)
        for od in client_models:
            for key, value in od.items():
                sums[key] += value

        # Calculate the average for each key
        averages = {key: value / count for key, value in sums.items()}

        # Convert the averages to an OrderedDict (optional)
        average_ordereddict = OrderedDict(averages)

        model.load_state_dict(average_ordereddict)
        testimages = torch.tensor(testimages, dtype=torch.float32)
        y_test = model(testimages)
        return y_test

    # print("Training complete and weights saved for each client model.")

# Configuration class
class Config:
    def __init__(self, dropout=0.5, learning_rate=0.001, num_epochs=50, batch_size=32):
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size

# Base Model class (assuming you have this implemented)
class Model(nn.Module):
    def __init__(self, config, name):
        super(Model, self).__init__()
        self.config = config
        self.name = name

# FeatureNN class (assuming you have this implemented)
class FeatureNN(nn.Module):
    def __init__(self, config, name, input_shape, num_units, feature_num):
        super(FeatureNN, self).__init__()
        self.config = config
        self.name = name
        self.input_shape = input_shape
        self.num_units = num_units
        self.feature_num = feature_num
        self.fc = nn.Linear(input_shape, num_units)

    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        return x

# NAM model definition
class NAM(Model):
    def __init__(self, config, name, *, num_inputs: int, num_units: int) -> None:
        super(NAM, self).__init__(config, name)
        self._num_inputs = num_inputs
        self.dropout = nn.Dropout(p=self.config.dropout)

        if isinstance(num_units, list):
            assert len(num_units) == num_inputs
            self._num_units = num_units
        elif isinstance(num_units, int):
            self._num_units = [num_units for _ in range(self._num_inputs)]

        self.feature_nns = nn.ModuleList([
            FeatureNN(config=config, name=f'FeatureNN_{i}', input_shape=1, num_units=self._num_units[i], feature_num=i)
            for i in range(num_inputs)
        ])

        self.output_layer = nn.Linear(sum(self._num_units), 3)  # 3 classes for Iris dataset
        self._bias = torch.nn.Parameter(data=torch.zeros(1))

    def calc_outputs(self, inputs: torch.Tensor) -> Sequence[torch.Tensor]:
        return [self.feature_nns[i](inputs[:, i:i+1]) for i in range(self._num_inputs)]

    def forward(self, inputs: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        individual_outputs = self.calc_outputs(inputs)
        conc_out = torch.cat(individual_outputs, dim=-1)
        dropout_out = self.dropout(conc_out)
        out = self.output_layer(dropout_out)
        return out, dropout_out

    def print_model_equation(self, feature_names):
        equation_terms = []
        feature_contributions = {}
        print("feature_names")
        print(feature_names)
        for i, fnn in enumerate(self.feature_nns):
            coefficients = fnn.fc.weight.data.flatten().tolist()
            intercepts = fnn.fc.bias.data.tolist()
            term = " + ".join([f"({coeff:.3f} * x_{feature_names[i]} + {intercept:.3f})" for coeff, intercept in zip(coefficients, intercepts)])
            equation_terms.append(term)
            feature_contributions[feature_names[i]] = sum(abs(c) for c in coefficients)
        equation = " + ".join(equation_terms) + f" + bias ({self._bias.item():.3f})"
        print(f"Model Equation: y = {equation}")
        model_equations.append(equation)

        # Determine feature interpretability based on coefficients
        interpretability = sorted(feature_contributions.items(), key=lambda x: x[1], reverse=True)
        print("\nFeature Contributions:")
        for feature, contribution in interpretability:
            print(f"{feature}: {contribution:.3f}")

        return interpretability[0][0],interpretability[-1][0]   # Return the feature with the highest contribution

n_clients = 3
# Load sample dataset
data = fetch_openml(name='wine-quality-red', version=1, as_frame=True, parser='liac-arff')
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target  # Ensure the target column is added to the DataFrame

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into n_clients
client_data = [df.iloc[i * len(df) // n_clients: (i + 1) * len(df) // n_clients] for i in range(n_clients)]

# Separate features and target for each client
clients_features = [client.drop(columns=['target']) for client in client_data]
clients_targets = [client['target'] for client in client_data]
clients_targets = [target.astype(int) for target in clients_targets]

# Separate features and target for each client
clients_features = [client.drop(columns=['target']) for client in client_data]

clients_features1 = {}
clients_features2 = {}
for i in range(n_clients):
    feature_columns = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol']

    df = clients_features[i].head()

    def adjust_target(value):
        return value - 3

    target = clients_targets[i].head()
    X = df[feature_columns].values
    y = target
    y = target.apply(adjust_target)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train = y_train.astype(int)
    y_train_np = y_train.to_numpy().astype(int)
    y_train_tensor = torch.tensor(y_train_np, dtype=torch.long)

    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test = y_test.astype(int)
    y_test_np = y_test.to_numpy()
    y_test_tensor = torch.tensor(y_test_np, dtype=torch.long)

    # Define the config
    config = Config(dropout=0.5, learning_rate=0.001, num_epochs=50, batch_size=32)

    # Instantiate the NAM model
    num_inputs = len(feature_columns)  # Number of features
    num_units = 10  # Number of units in the hidden layer
    nam_model = NAM(config=config, name='NAM_Model', num_inputs=num_inputs, num_units=num_units)

    # Training function
    def train(model, X_train, y_train, config):
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
        model.train()
        for epoch in range(config.num_epochs):
            outputs = fed_model(X_test_tensor)
            optimizer.zero_grad()
            y_test = torch.tensor(y_test_tensor, dtype=torch.float32)
            loss = criterion(outputs, y_test_tensor)
            loss.backward()
            optimizer.step()
            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch + 1}/{config.num_epochs}], Loss: {loss.item():.4f}')
        return model

    # Evaluation function
    def evaluate(model, X_test, y_test):
        model.eval()
        with torch.no_grad():
            outputs, _ = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == y_test).sum().item() / y_test.size(0)
            print(f'Accuracy: {accuracy * 100:.2f}%')

    # Train the model
    trained_model = train(nam_model, X_train_tensor, y_train_tensor, config)

    # Evaluate the model
    evaluate(trained_model, X_test_tensor, y_test_tensor)

    # Print the model equation and get the most contributing feature
    a,b = trained_model.print_model_equation(feature_columns)
    most_contributing_feature = a
    clients_features1[i] = most_contributing_feature
    least_contributing_feature = b
    clients_features2[i] = least_contributing_feature
    print(f"\nMost contributing feature for client's output {i}: {most_contributing_feature}")

# pip install openai==0.28

!pip install openai --upgrade

# openai migrate
# from main_workshop import clients_features1, clients_features2


<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test_tensor, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.cl

Epoch [10/50], Loss: 16.1449


<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test_tensor, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.cl

Epoch [20/50], Loss: 5.9893


<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test_tensor, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.cl

...

<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test_tensor, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.cl

Epoch [40/50], Loss: 3.8165


<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test_tensor, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:140: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  testimages = torch.tensor(testimages, dtype=torch.float32)
<ipython-input-2-5fc38d3af00a>:296: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.cl

Epoch [50/50], Loss: 3.9126
Accuracy: 0.00%
feature_names
['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol']
Model Equation: y = (0.322 * x_fixed_acidity + -0.344) + (-0.613 * x_fixed_acidity + -0.255) + (0.751 * x_fixed_acidity + 0.805) + (-0.687 * x_fixed_acidity + -0.937) + (-0.250 * x_fixed_acidity + 0.080) + (0.640 * x_fixed_acidity + -0.296) + (0.779 * x_fixed_acidity + 0.008) + (0.473 * x_fixed_acidity + -0.303) + (0.235 * x_fixed_acidity + 0.419) + (-0.014 * x_fixed_acidity + -0.255) + (0.719 * x_volatile_acidity + -0.404) + (-0.116 * x_volatile_acidity + -0.729) + (0.490 * x_volatile_acidity + 0.077) + (-0.915 * x_volatile_acidity + 0.871) + (-0.226 * x_volatile_acidity + 0.446) + (0.987 * x_volatile_acidity + 0.352) + (-0.139 * x_volatile_acidity + 0.757) + (-0.860 * x_volatile_acidity + -0.387) + (0.408 * x_volatile_acidity + -0.483) + (0.844 * x_volatile_

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [4]:
pip install openai==0.28


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.42.0
    Uninstalling openai-1.42.0:
      Successfully uninstalled openai-1.42.0


In [9]:
pip install cohere

In [11]:
import openai
import cohere

# Set your GPT API key here
openai.api_key = "sk-RWK4Bt75LGo2pIdI3Nne0Al7JI6rky9WVRDl5bcq52T3BlbkFJqUrncWB7_-mU3XR-vPQZHqvnu4DaI1n0gM5l6CT1IA"

def generate_feature_report(client_name, high_contrib_feature, least_contrib_feature):
    prompt = f"""
    Generate a report for {client_name}:
    1. High Contributing Feature: {high_contrib_feature}
    - Analysis: Why this feature contributes highly to the model's performance.
    - Improvement Strategy: Ways to further enhance the impact of this feature.

    2. Least Contributing Feature: {least_contrib_feature}
    - Analysis: Why this feature has a low contribution to the model's performance.
    - Improvement Strategy: How to address the inefficiency or whether to consider removing this feature.
    """

    # Initialize the Cohere client
    co = cohere.Client('Isi9ZkalXhCu0ck8masw8vsqn11lkqhVaMOEU7Bq')  # Replace with your Cohere API key

    # Generate the report using the Cohere generate function
    response = co.generate(
        model='command-xlarge-nightly',  # Model choice depending on availability
        prompt=prompt,
        max_tokens=300  # Adjust based on the expected length of the report
    )

    # Extract and return the response
    return response.generations[0].text.strip()

def generate_full_report(clients_features):
    report = "### Analysis Report on Client Feature Contribution and Improvement Strategies\n\n"
    report += "---\n\n"

    for client in clients_features:
        client_name = client["name"]
        high_contrib_feature = client["high_contrib_feature"]
        least_contrib_feature = client["least_contrib_feature"]

        client_report = generate_feature_report(client_name, high_contrib_feature, least_contrib_feature)
        report += f"#### {client_name}\n"
        report += client_report
        report += "\n\n---\n\n"

    return report

# Example usage with dynamic client input
def main():
    n = 3  # Number of clients
    clients_features = []
    for i in range(n):
        name = i  # Example client name, replace with actual names
        high_contrib_feature = clients_features1[i]  # Example high contributing feature
        least_contrib_feature = clients_features2[i]  # Example least contributing feature

        clients_features.append({
            "name": name,
            "high_contrib_feature": high_contrib_feature,
            "least_contrib_feature": least_contrib_feature
        })

    # Generate the full report
    full_report = generate_full_report(clients_features)

    # Save the report to a file
    with open("client_feature_report.txt", "w") as report_file:
        report_file.write(full_report)

    print("Report generated and saved to 'client_feature_report.txt'")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Report generated and saved to 'client_feature_report.txt'
